# Kinematic generator
This notebook converts a Denavit Hartenberg table to its direct kinematic symbolic calculation and its jacobian

In [1]:
from cmath import pi
import numpy as np
import sympy
from sympy.abc import x,y,z,a,d

# import sys
# sys.set_int_max_str_digits(0)

## Joints
This classes define a single joint in symbolic representation and stores its type

In [2]:
from enum import Enum

class JointType(Enum):
    ROTATIONAL = 0
    PRISMATIC = 1

class Joint:
    def __init__(self, symbol: sympy.Symbol, jointType: JointType):
        self.type = jointType
        self.symbol = symbol

## Denavit Hartenberg
Symbolic Denavit table 

In [3]:
class Denavit:
    def __init__(self):
        self.theta = sympy.Symbol('theta')
        self.alfa = sympy.Symbol('alpha') 
        self.d = sympy.Symbol('d')
        self.a = sympy.Symbol('a')

        cosT = sympy.cos(self.theta)
        sinT = sympy.sin(self.theta)
        cosA = sympy.cos(self.alfa)
        sinA = sympy.sin(self.alfa)
        
        self.denavit = sympy.Matrix([
            [cosT,  -sinT*cosA, sinT*sinA,  self.a*cosT ],
            [sinT,  cosT*cosA,  -cosT*sinA, self.a*sinT ],
            [0,     sinA,       cosA,       self.d      ],
            [0,     0,          0,          1           ]
        ])

Denavit().denavit

Matrix([
[cos(theta), -sin(theta)*cos(alpha),  sin(alpha)*sin(theta), a*cos(theta)],
[sin(theta),  cos(alpha)*cos(theta), -sin(alpha)*cos(theta), a*sin(theta)],
[         0,             sin(alpha),             cos(alpha),            d],
[         0,                      0,                      0,            1]])

## Denavit row of the table
This class sets a single row of the Denavit Hartenberg table and computes the transformation matrixes, both in symbolic and lambda representations

In [4]:
class DenavitRow(Denavit):
    def __init__(self, dh_theta:float, dh_d:float, dh_a:float, dh_alfa:float, joint:Joint = None):
        super().__init__()
        self.dhParams = (dh_theta,dh_d,dh_a,dh_alfa)
        self.joint = joint

        if joint is not None:
            if joint.type is JointType.ROTATIONAL:
                T = self.denavit.subs(self.theta, self.theta+joint.symbol)
            elif joint.type is JointType.PRISMATIC:
                T = self.denavit.subs(self.d, self.d+joint.symbol)
            else:
                raise ValueError("Invalid joint type")
        else:
            T = self.denavit
        
        T = T.subs(self.theta,dh_theta)
        T = T.subs(self.d,dh_d)
        T = T.subs(self.a,dh_a)
        T = T.subs(self.alfa,dh_alfa)
        self.TransformSym = T

    def get(self):
        return (self.theta,self.d, self.a, self.alfa, self.joint.value)

    def eval(self, jointVal:float):
        self.joint.value = jointVal
        return self.Transform(*self.get())
    
    def getRotationSym(self):
        return self.TransformSym[0:3,0:3]
    
    def getTranslationSym(self):
        return self.TransformSym[0:3,3]

## Denavit Direct kinematics
This class takes an ordered list of the Denavit rows and computes the total kinematics in symbolics and lambda

In [5]:
class DenavitDK:
    def __init__(self,denavitRows) -> None:
        self.directLambdaLambdaTransformSym = sympy.Matrix(np.eye(4))
        self.jointsSym = []
        # Compute direct kinematis and record joint symbols
        for T in denavitRows:
            if T.joint is not None:
                self.jointsSym.append(T.joint.symbol)
            self.directLambdaLambdaTransformSym = self.directLambdaLambdaTransformSym*T.TransformSym
        # Clean almost zero values
        self.directLambdaLambdaTransformSym = sympy.nsimplify(self.directLambdaLambdaTransformSym,tolerance=1e-15,rational=True)
        # self.directLambdaLambdaTransformSym = sympy.simplify(self.directLambdaLambdaTransformSym)
        # Set joints number
        self.jointsNum = len(self.jointsSym)
        # Set joints array
        self.jointsSym = sympy.Matrix([q for q in self.jointsSym])
        # Set lambda for direct transform
        self.directLambdaLambdaTransform = sympy.lambdify(self.jointsSym,self.directLambdaLambdaTransformSym)
        # Calculate jacobian
        self._jacobian(denavitRows)

    
    def eval(self, jointVal:list):
        return self.directLambdaLambdaTransform(*jointVal)

    def getRotationSym(self):
        return self.directLambdaLambdaTransformSym[0:3,0:3]
    
    def getTranslationSym(self):
        return self.directLambdaLambdaTransformSym[0:3,3]
    
    def _jacobian(self, denavitRows):
        Transform = sympy.eye(4)
        z_vector = sympy.Matrix([[0],[0],[1]])
        J = []
        for T in denavitRows:
            if T.joint is None:
                pass
            elif T.joint.type is JointType.PRISMATIC:
                translation = Transform[0:3,0:3]*z_vector
                J_row = sympy.Matrix([translation,sympy.zeros(3,1)]).T
            elif T.joint.type is JointType.ROTATIONAL:
                rotation    = Transform[0:3,0:3]*z_vector
                translation = rotation.cross(self.getTranslationSym()-Transform[0:3,3])
                J_row = sympy.Matrix([translation,rotation]).T
            else:
                raise ValueError("Invalid joint type")
            Transform = Transform*T.TransformSym

            J.append(J_row)
        self.jacobian = sympy.Matrix(J).T
        self.jacobian = sympy.nsimplify(self.jacobian,tolerance=1e-10,rational=True)
        return self.jacobian

    def genCCode(self, filename:str, simplify:bool = False):
        from sympy.utilities.codegen import codegen

        if simplify:
            symDK = sympy.simplify(self.directLambdaLambdaTransformSym)
            symJ = sympy.simplify(self.jacobian)
        else:
            symDK = self.directLambdaLambdaTransformSym
            symJ = self.jacobian

        [(c_name, c_code_dk), _] = codegen(('directKin', symDK), "C99", filename, header=False, empty=False)
        [(c_name, c_code_j), _] = codegen(('jacobian', symJ), "C99", filename, header=False, empty=False)
        with open(c_name,'w+') as c_file:
            c_file.write(c_code_dk)
            c_file.write(c_code_j)



# Examples

## 3 DOF Cartesian 
A 3 DOF Cartesian robot with 3 prismatic joints from [The Ultimate Guide to Jacobian Matrices for Robotics](https://automaticaddison.com/the-ultimate-guide-to-jacobian-matrices-for-robotics/)

In [6]:
T1 = DenavitRow(pi/2,   0, 0, pi/2,  Joint(sympy.Symbol('q_1'),JointType.PRISMATIC))
T2 = DenavitRow(pi/2,   0, 0, -pi/2, Joint(sympy.Symbol('q_2'),JointType.PRISMATIC))
T3 = DenavitRow(0,      0, 0, 0,     Joint(sympy.Symbol('q_3'),JointType.PRISMATIC))

T_cartesian = DenavitDK((T1,T2,T3))
T_cartesian.directTransformSym

Matrix([
[0, -1,  0,  q_2],
[0,  0, -1, -q_3],
[1,  0,  0,  q_1],
[0,  0,  0,    1]])

In [7]:
T_cartesian.jacobian

Matrix([
[0, 1,  0],
[0, 0, -1],
[1, 0,  0],
[0, 0,  0],
[0, 0,  0],
[0, 0,  0]])

In [8]:
cartesian = T_cartesian.jacobian * T_cartesian.jointsSym
cartesian

Matrix([
[ q_2],
[-q_3],
[ q_1],
[   0],
[   0],
[   0]])

In [9]:
from sympy.abc import x,y,z
ox,oy,oz = sympy.symbols('omega_x omega_y omega_z')
cartesian = sympy.Matrix([
    x, y, z, ox, oy, oz
])

joints_spd = T_cartesian.jacobian.T*cartesian
joints_spd

Matrix([
[ z],
[ x],
[-y]])

In [10]:
joint_values = (0, 0, 0)
T_cartesian.eval(joint_values)

array([[ 0, -1,  0,  0],
       [ 0,  0, -1,  0],
       [ 1,  0,  0,  0],
       [ 0,  0,  0,  1]])

## 3 DOF Articulated Robot
A 3 DOF Articulated robot with 3 rotational joints from [The Ultimate Guide to Jacobian Matrices for Robotics](https://automaticaddison.com/the-ultimate-guide-to-jacobian-matrices-for-robotics/)

In [11]:
T1 = DenavitRow(0,  1, 0,   pi/2, Joint(sympy.Symbol('q_1'),JointType.ROTATIONAL))
T2 = DenavitRow(0,  0,  1,  0,    Joint(sympy.Symbol('q_2'),JointType.ROTATIONAL))
T3 = DenavitRow(0,  0,  1,  0,    Joint(sympy.Symbol('q_3'),JointType.ROTATIONAL))

T_articular3 = DenavitDK((T1,T2,T3))
T_articular3.directTransformSym

Matrix([
[-sin(q_2)*sin(q_3)*cos(q_1) + cos(q_1)*cos(q_2)*cos(q_3), -sin(q_2)*cos(q_1)*cos(q_3) - sin(q_3)*cos(q_1)*cos(q_2),  sin(q_1), -sin(q_2)*sin(q_3)*cos(q_1) + cos(q_1)*cos(q_2)*cos(q_3) + cos(q_1)*cos(q_2)],
[-sin(q_1)*sin(q_2)*sin(q_3) + sin(q_1)*cos(q_2)*cos(q_3), -sin(q_1)*sin(q_2)*cos(q_3) - sin(q_1)*sin(q_3)*cos(q_2), -cos(q_1), -sin(q_1)*sin(q_2)*sin(q_3) + sin(q_1)*cos(q_2)*cos(q_3) + sin(q_1)*cos(q_2)],
[                   sin(q_2)*cos(q_3) + sin(q_3)*cos(q_2),                   -sin(q_2)*sin(q_3) + cos(q_2)*cos(q_3),         0,                         sin(q_2)*cos(q_3) + sin(q_2) + sin(q_3)*cos(q_2) + 1],
[                                                       0,                                                        0,         0,                                                                            1]])

In [12]:
T_articular3.jacobian

Matrix([
[ sin(q_1)*sin(q_2)*sin(q_3) - sin(q_1)*cos(q_2)*cos(q_3) - sin(q_1)*cos(q_2),                                                                                                                      -(sin(q_2)*cos(q_3) + sin(q_2) + sin(q_3)*cos(q_2))*cos(q_1),                                                                                         -(sin(q_2)*cos(q_3) + sin(q_3)*cos(q_2))*cos(q_1)],
[-sin(q_2)*sin(q_3)*cos(q_1) + cos(q_1)*cos(q_2)*cos(q_3) + cos(q_1)*cos(q_2),                                                                                                                      -(sin(q_2)*cos(q_3) + sin(q_2) + sin(q_3)*cos(q_2))*sin(q_1),                                                                                         -(sin(q_2)*cos(q_3) + sin(q_3)*cos(q_2))*sin(q_1)],
[                                                                           0, (-sin(q_1)*sin(q_2)*sin(q_3) + sin(q_1)*cos(q_2)*cos(q_3) + sin(q_1)*cos(q_2))*sin(q_1) + (-sin(q_2)*sin(q_3)*cos(q_

In [13]:
cartesian = T_articular3.jacobian * T_articular3.jointsSym
cartesian

Matrix([
[                                                                                                                               q_1*(sin(q_1)*sin(q_2)*sin(q_3) - sin(q_1)*cos(q_2)*cos(q_3) - sin(q_1)*cos(q_2)) - q_2*(sin(q_2)*cos(q_3) + sin(q_2) + sin(q_3)*cos(q_2))*cos(q_1) - q_3*(sin(q_2)*cos(q_3) + sin(q_3)*cos(q_2))*cos(q_1)],
[                                                                                                                              q_1*(-sin(q_2)*sin(q_3)*cos(q_1) + cos(q_1)*cos(q_2)*cos(q_3) + cos(q_1)*cos(q_2)) - q_2*(sin(q_2)*cos(q_3) + sin(q_2) + sin(q_3)*cos(q_2))*sin(q_1) - q_3*(sin(q_2)*cos(q_3) + sin(q_3)*cos(q_2))*sin(q_1)],
[q_2*((-sin(q_1)*sin(q_2)*sin(q_3) + sin(q_1)*cos(q_2)*cos(q_3) + sin(q_1)*cos(q_2))*sin(q_1) + (-sin(q_2)*sin(q_3)*cos(q_1) + cos(q_1)*cos(q_2)*cos(q_3) + cos(q_1)*cos(q_2))*cos(q_1)) + q_3*((-sin(q_1)*sin(q_2)*sin(q_3) + sin(q_1)*cos(q_2)*cos(q_3))*sin(q_1) + (-sin(q_2)*sin(q_3)*cos(q_1) + cos(q_1)*cos(q_2)*cos(q_3))*cos(

In [14]:
from sympy.abc import x,y,z
ox,oy,oz = sympy.symbols('omega_x omega_y omega_z')
cartesian = sympy.Matrix([
    x, y, z, ox, oy, oz
])

joints_spd = T_articular3.jacobian.T*cartesian
joints_spd

Matrix([
[                                                                                                                                                                               omega_z + x*(sin(q_1)*sin(q_2)*sin(q_3) - sin(q_1)*cos(q_2)*cos(q_3) - sin(q_1)*cos(q_2)) + y*(-sin(q_2)*sin(q_3)*cos(q_1) + cos(q_1)*cos(q_2)*cos(q_3) + cos(q_1)*cos(q_2))],
[omega_x*sin(q_1) - omega_y*cos(q_1) - x*(sin(q_2)*cos(q_3) + sin(q_2) + sin(q_3)*cos(q_2))*cos(q_1) - y*(sin(q_2)*cos(q_3) + sin(q_2) + sin(q_3)*cos(q_2))*sin(q_1) + z*((-sin(q_1)*sin(q_2)*sin(q_3) + sin(q_1)*cos(q_2)*cos(q_3) + sin(q_1)*cos(q_2))*sin(q_1) + (-sin(q_2)*sin(q_3)*cos(q_1) + cos(q_1)*cos(q_2)*cos(q_3) + cos(q_1)*cos(q_2))*cos(q_1))],
[                                                              omega_x*sin(q_1) - omega_y*cos(q_1) - x*(sin(q_2)*cos(q_3) + sin(q_3)*cos(q_2))*cos(q_1) - y*(sin(q_2)*cos(q_3) + sin(q_3)*cos(q_2))*sin(q_1) + z*((-sin(q_1)*sin(q_2)*sin(q_3) + sin(q_1)*cos(q_2)*cos(q_3))*sin(q_1) + (-sin(q_2

In [15]:
joint_values = (0, 0, 0)
T_articular3.eval(joint_values)

array([[ 1., -0.,  0.,  2.],
       [ 0., -0., -1.,  0.],
       [ 0.,  1.,  0.,  1.],
       [ 0.,  0.,  0.,  1.]])

## 6 DOF articular
A 6 DOF articular robot that follows the following scheme: 
<!-- ![axes](axes_diagram.png) -->
<img src="axes_diagram.png" alt="axes_diagram" width="400"/>

In [16]:
L = [0.2, 1, 1, 0.2]
T1 = DenavitRow(0,      L[0],   0,      pi/2,   Joint(sympy.Symbol('q_1'),JointType.ROTATIONAL))
T2 = DenavitRow(pi/2,   0,      L[1],   0,      Joint(sympy.Symbol('q_2'),JointType.ROTATIONAL))
T3 = DenavitRow(pi,     0,      0,      pi/2,   Joint(sympy.Symbol('q_3'),JointType.ROTATIONAL))
T4 = DenavitRow(pi,     L[2],   0,      pi/2,   Joint(sympy.Symbol('q_4'),JointType.ROTATIONAL))
T5 = DenavitRow(pi,     0,      0,      pi/2,   Joint(sympy.Symbol('q_5'),JointType.ROTATIONAL))
T6 = DenavitRow(0,      L[3],   0,      0,      Joint(sympy.Symbol('q_6'),JointType.ROTATIONAL))

T_articular6 = DenavitDK((T1,T2,T3,T4,T5,T6))
T_articular6.directTransformSym

Matrix([
[(((-sin(q_2 + 884279719003555/562949953421312)*sin(q_3 + 884279719003555/281474976710656)*cos(q_1) + cos(q_1)*cos(q_2 + 884279719003555/562949953421312)*cos(q_3 + 884279719003555/281474976710656))*cos(q_4 + 884279719003555/281474976710656) + sin(q_1)*sin(q_4 + 884279719003555/281474976710656))*cos(q_5 + 884279719003555/281474976710656) + (sin(q_2 + 884279719003555/562949953421312)*cos(q_1)*cos(q_3 + 884279719003555/281474976710656) + sin(q_3 + 884279719003555/281474976710656)*cos(q_1)*cos(q_2 + 884279719003555/562949953421312))*sin(q_5 + 884279719003555/281474976710656))*cos(q_6) + ((-sin(q_2 + 884279719003555/562949953421312)*sin(q_3 + 884279719003555/281474976710656)*cos(q_1) + cos(q_1)*cos(q_2 + 884279719003555/562949953421312)*cos(q_3 + 884279719003555/281474976710656))*sin(q_4 + 884279719003555/281474976710656) - sin(q_1)*cos(q_4 + 884279719003555/281474976710656))*sin(q_6), -(((-sin(q_2 + 884279719003555/562949953421312)*sin(q_3 + 884279719003555/281474976710656)*cos(q_

In [17]:
T_articular6.jacobian

Matrix([
[-((-sin(q_1)*sin(q_2 + 884279719003555/562949953421312)*sin(q_3 + 884279719003555/281474976710656) + sin(q_1)*cos(q_2 + 884279719003555/562949953421312)*cos(q_3 + 884279719003555/281474976710656))*cos(q_4 + 884279719003555/281474976710656) - sin(q_4 + 884279719003555/281474976710656)*cos(q_1))*sin(q_5 + 884279719003555/281474976710656)/5 + (sin(q_1)*sin(q_2 + 884279719003555/562949953421312)*cos(q_3 + 884279719003555/281474976710656) + sin(q_1)*sin(q_3 + 884279719003555/281474976710656)*cos(q_2 + 884279719003555/562949953421312))*cos(q_5 + 884279719003555/281474976710656)/5 - sin(q_1)*sin(q_2 + 884279719003555/562949953421312)*cos(q_3 + 884279719003555/281474976710656) - sin(q_1)*sin(q_3 + 884279719003555/281474976710656)*cos(q_2 + 884279719003555/562949953421312) - sin(q_1)*cos(q_2 + 884279719003555/562949953421312),                                                                                                                                                                 

In [18]:
# T_articular6.genCCode("6dof",True)